In [3]:
import pandas as pd

col = ['Reviews', 'Feedback']
data = pd.read_csv('/Users/ferdianadham/Downloads/TugasDL/model/data_train.csv', names=col)

data.head()

,Reviews,Feedback
0,Makanan terlalu asin,Kurangi garam
1,Makanan hambar,Tambah gula dan garam
2,Makanan kurang asin,Tambah garam
3,Terlalu manis,Kurangi gula
4,Tidak ada rasa,Tambah gula dan garam


In [5]:
data = data.map(lambda x : x.lower() if isinstance(x, str) else x)

In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()

stemmer = factory.create_stemmer()

data['Reviews'] = [stemmer.stem(doc) for doc in data['Reviews']]

data.head()


,Reviews,Feedback
0,makan terlalu asin,kurangi garam
1,makan hambar,tambah gula dan garam
2,makan kurang asin,tambah garam
3,terlalu manis,kurangi gula
4,tidak ada rasa,tambah gula dan garam


In [11]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

data['label'] = label_encoder.fit_transform(data['Feedback'])


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=500, oov_token="<OOV>")
tokenizer.fit_on_texts(data['Reviews'])
sequences = tokenizer.texts_to_sequences(data['Reviews'])
padded_sequences = pad_sequences(sequences, maxlen=20)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=500, output_dim=64),
    LSTM(64, return_sequences=False),
    Dense(64, activation="relu"),
    Dense(len(label_encoder.classes_), activation="softmax")
])


In [16]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, data["label"], test_size=0.2
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    batch_size=16
)

Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.0233 - loss: 3.5833 - val_accuracy: 0.0476 - val_loss: 3.5728
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.1674 - loss: 3.5622 - val_accuracy: 0.0476 - val_loss: 3.5571
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1854 - loss: 3.5414 - val_accuracy: 0.0476 - val_loss: 3.5360
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1577 - loss: 3.5049 - val_accuracy: 0.0476 - val_loss: 3.4825
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1828 - loss: 3.3956 - val_accuracy: 0.0476 - val_loss: 3.4855
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1517 - loss: 3.2970 - val_accuracy: 0.0476 - val_loss: 3.5408
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1621 - loss: 3.2494 - val_accuracy: 0.0476 - val_loss: 3.4441
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1473 - loss: 3.2012 - val_accuracy: 0.1429 - val_loss: 3.4258


In [32]:
import numpy as np
contoh_ulasan = ["kurang asin"]
seq = tokenizer.texts_to_sequences(contoh_ulasan)
padded = pad_sequences(seq, maxlen=20)

prediksi = model.predict(padded)
solusi = label_encoder.inverse_transform([np.argmax(prediksi)])
print(solusi)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
['tambah garam']
